In [ ]:
from __future__ import print_function    # (at top of module)
import warnings
#warnings.filterwarnings('always')
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]
from matplotlib.pyplot import figure
import math
import seaborn as sns
import config
import requests
import urllib

fileName = "data_600_entries.csv"

In [ ]:
data = pd.read_csv(fileName ,encoding='utf-8')
data = data.drop_duplicates(subset=['song_id'], keep='first')
data = data.drop_duplicates(subset=['song_title'], keep='first')
data = data[data.popularity > 50]
print("Number of entries in original data after cleaning: " + str(len(data.index)))

In [ ]:
# Build the query from the data
temp_query = ""
youtube_view_counts = []
youtube_video_titles = []
number_of_reaction_videos = 0

# For each row in the dataframe
for index, row in data.iterrows():
    
    # Build the query as song_title + artists
    temp_query += row['song_title'].replace(',', "") + " " + row['artist'].replace('[', "").replace(']', "").replace(',', "")
    print("===============")
    print(temp_query)
    
    params_for_query = {"part": "snippet",
                        "maxResults": 5,
                        "order": "relevance",
                        "publishedAfter": "2017-01-01T00:00:00Z",
                        "pageToken": "",
                        "q": temp_query,
                        "key": config.youtube_client_secret,
                        "type": "video",
                        }
    url1 = "https://www.googleapis.com/youtube/v3/search"
    
    # Get video results, select most relevant video (first one)
    # TODO: Still some matching problems - how to get the most relevant video with the most views from official sources?
    page_query = requests.request(method="get", url=url1, params=params_for_query)
    j_results_query = json.loads(page_query.text)
    video_item = j_results_query['items'][0]
    print("==============")
    
    # TOFIX: Hacky way of removing reaction videos - they sometimes have the full song title in the description and 
    #        are considered as false positives - if it has react in the title, get the next one
    # A way to fix the matching problem could be to iterate over all the results and get the video that has the most views
    # from the top 5 most relevant ones - could significantly increase processing time
    if video_item['snippet']['title'].lower().find("react") > 0:
        print("REACT TITLE: ",video_item['snippet']['title'])
        number_of_reaction_videos += 1
        video_item = j_results_query['items'][1]
    
    # Print the video title of the first video to match the query string
    print("Video title: ", video_item['snippet']['title'])
    video_title = video_item['snippet']['title']
    
    # Keep titles column in list
    youtube_video_titles.append(video_title)
    
    # Get the video ID of the first video to match the query string
    print("Video id: ", video_item['id']['videoId'])
    video_id = video_item['id']['videoId']
        
    # Next, using this video ID we need to get the view count of that video
    params_for_stats = {"part": "statistics",
                        "id": video_id,
                        "key": config.youtube_client_secret,
                       }
    url2 = "https://www.googleapis.com/youtube/v3/videos"
    page_stats = requests.request(method="get", url=url2, params=params_for_stats)
    j_results_stats = json.loads(page_stats.text)
    
    # Check if problem with retrieving statistics
    if len(j_results_stats['items']) > 0:
        # Get the view count
        print("Video view count: ", j_results_stats['items'][0]['statistics']['viewCount'])
        view_count  = j_results_stats['items'][0]['statistics']['viewCount']
        
        # Keep count in list
        youtube_view_counts.append(view_count)
    else:
        youtube_view_counts.append(0)
        
    temp_query = ""
    
# Add the two new columns to the data
data['youtube_view_count'] = youtube_view_counts
data['youtube_video_title'] = youtube_video_titles

print()
print("Number of reaction videos found: ", number_of_reaction_videos)
print()
print("Done getting data.")

In [ ]:
data.to_csv("data_try.csv", encoding="utf-8", header=True, index=False)